## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


## Camera Calibration

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(100)

cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

## Apply Undistortion

In [65]:
def display_img(img):
    cv2.imshow("test", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def warp_binary_img(img, sobelx_thresh_min = 40, sobelx_thresh_max = 100, s_thresh_min=170, s_thresh_max = 255):
    # undistort image
    undistort = cv2.undistort(img, mtx, dist, None, mtx)
    # grayscale image
    gray = cv2.cvtColor(undistort, cv2.COLOR_BGR2GRAY)
    
    #sobel x
    sobelx = np.abs(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    scaled_sobel = np.uint8(255*sobelx/np.max(sobelx))
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sobelx_thresh_min) & (scaled_sobel <= sobelx_thresh_max)] = 1
    
    # threshold color channel
    hls = cv2.cvtColor(undistort, cv2.COLOR_BGR2HLS)
    s_channel = hls[:, :, 2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1
    
    # combining thresholded images
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    # create binary image
    binary_image = combined_binary * 255
    # create perspective transform
    src = np.float32([[535, 500], [790, 500], [1105, 700], [280, 700]])
    dst = np.float32([[200, 0], [1000, 0], [1000, 680], [200, 680]])
    M = cv2.getPerspectiveTransform(src, dst)
    warped_binary = cv2.warpPerspective(binary_image, M, binary_image.shape[::-1], flags=cv2.INTER_LINEAR)
    
    display_img(warped_binary)
    return warped_binary

    
test_img = cv2.imread("./test_images/test1.jpg")
warped_binary = warp_binary_img(test_img)